In [2]:
import pathlib
from importlib import reload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr

In [3]:
ds = xr.open_dataset("tests/data/test_SPI1_ERA5_ChinaXinjiang_sample.nc")
ds

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject


<xarray.Dataset> Size: 634kB
Dimensions:  (time: 492, lon: 16, lat: 10)
Coordinates:
  * time     (time) datetime64[ns] 4kB 1960-01-31 1960-02-29 ... 2000-12-31
  * lon      (lon) float64 128B 73.5 75.0 76.5 78.0 79.5 ... 91.5 93.0 94.5 96.0
  * lat      (lat) float64 80B 35.25 36.75 38.25 39.75 ... 45.75 47.25 48.75
Data variables:
    SPI1     (time, lat, lon) float64 630kB ...
Attributes:
    CDI:          Climate Data Interface version 2.4.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Dec 23 17:09:12 2024: cdo selyear,1960/2000 SPI1_ERA5_C...
    CDO:          Climate Data Operators version 2.4.0 (https://mpimet.mpg.de...

# Stack locations

In [4]:
import dominosee.utils
from dominosee.utils import stack_lonlat
reload(dominosee.utils)

<module 'dominosee.utils' from '/home/hmwang-cli/PycharmProjects/dominosee-dev/dominosee/utils.py'>

In [5]:
da = stack_lonlat(ds.SPI1)
da

<xarray.DataArray 'SPI1' (time: 492, location: 160)> Size: 630kB
array([[-0.17908246, -0.34275251, -0.53795448, ...,  1.24861188,
         0.71117709,  1.15230826],
       [-0.91450245, -1.08202704, -1.32470199, ...,  2.10458992,
         1.84203189,  1.37324636],
       [ 1.38179004,  1.48488332,  1.38575002, ..., -0.92376469,
        -1.13522819, -0.66207943],
       ...,
       [-0.33751102, -0.26478241, -0.19772469, ...,  1.56139293,
         2.09853221,  2.02687496],
       [ 0.49602831,  0.49755386,  0.65215184, ...,  0.93239077,
         0.85523871,  0.59302988],
       [ 0.01624789, -0.03762084, -0.02580197, ..., -0.02435134,
         0.45573657,  0.96576791]])
Coordinates:
  * time      (time) datetime64[ns] 4kB 1960-01-31 1960-02-29 ... 2000-12-31
  * location  (location) object 1kB MultiIndex
  * lat       (location) float64 1kB 35.25 35.25 35.25 ... 48.75 48.75 48.75
  * lon       (location) float64 1kB 73.5 75.0 76.5 78.0 ... 91.5 93.0 94.5 96.0
Attributes:
    calibration_period:  
    freq:                ME
    window:              1
    scipy_dist:          gamma
    method:              APP
    group:               time.month
    time_indexer:

# Extract event layers

In [6]:
import dominosee.eventorize
reload(dominosee.eventorize)
from dominosee.eventorize import get_events, events_to_layer

In [7]:
da_drought = get_events(da, threshold=-1.0, extreme="below", outname="drought")
da_pluvial = get_events(da, threshold=1.0, extreme="above", outname="pluvial")

In [11]:
ds_layer = events_to_layer((da_drought, da_pluvial))
ds_layer

<xarray.Dataset> Size: 165kB
Dimensions:   (time: 492, location: 160)
Coordinates:
  * time      (time) datetime64[ns] 4kB 1960-01-31 1960-02-29 ... 2000-12-31
  * location  (location) object 1kB MultiIndex
  * lat       (location) float64 1kB 35.25 35.25 35.25 ... 48.75 48.75 48.75
  * lon       (location) float64 1kB 73.5 75.0 76.5 78.0 ... 91.5 93.0 94.5 96.0
Data variables:
    drought   (time, location) bool 79kB False False False ... False False False
    pluvial   (time, location) bool 79kB False False False ... False False False

# Calculate eca

In [15]:
import dominosee.eca
reload(dominosee.eca)
from dominosee.eca import get_eca_window, eca_dataset

In [17]:
da_drought_precwindow, da_drought_trigwindow = get_eca_window(ds_layer.drought)
da_drought_precwindow

<xarray.DataArray 'drought' (location: 160, time: 492)> Size: 79kB
array([[False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True,  True, False],
       [ True,  True,  True, ..., False, False, False],
       [False, False, False, ...,  True,  True, False]])
Coordinates:
  * time      (time) datetime64[ns] 4kB 1960-01-31 1960-02-29 ... 2000-12-31
  * location  (location) object 1kB MultiIndex
  * lat       (location) float64 1kB 35.25 35.25 35.25 ... 48.75 48.75 48.75
  * lon       (location) float64 1kB 73.5 75.0 76.5 78.0 ... 91.5 93.0 94.5 96.0
Attributes:
    long_name:    Precursor Window
    units:        boolean
    description:  Window for precursor event identification
    eca_params:   {"delt": 2, "sym": true, "tau": 0}